In [1]:
import random
import os
import ssl
import cv2
import numpy as np
import imageio
from IPython import display
from urllib import request
import re
import tempfile
import math
from tqdm.notebook import tqdm

In [2]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [3]:
# fetch videos from UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
unverified_context = ssl._create_unverified_context()
UCF_DATA_FOLDER = r"C:\Users\hikar\Documents\UCF_101"

def list_ucf_videos():
    global _VIDEO_LIST
    if not _VIDEO_LIST:
        index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
        videos = re.findall("(v_[\w_]+\.avi)", index)
        _VIDEO_LIST = sorted(set(videos))
    return list(_VIDEO_LIST)

def fetch_ucf_video(video):
    action_dir = video.split('_')[1]
    if action_dir == "HandstandPushups":
      action_dir = "HandStandPushups"
      text_list = video.split('_')
      base_video_path = text_list[0]+ '_' + action_dir + '_' + text_list[2] + '_' + text_list[3]
      video_path = os.path.join(UCF_DATA_FOLDER,action_dir,base_video_path)
      return video_path
    
    video_path = os.path.join(UCF_DATA_FOLDER,action_dir,video)
    return video_path
  
def path_modify(video):
  action_dir = video.split('_')[1]
  if action_dir == "HandstandPushups":
      action_dir = "HandStandPushups"
      text_list = video.split('_')
      base_video_path = text_list[0]+ '_' + action_dir + '_' + text_list[2] + '_' + text_list[3]
      print(base_video_path)
      return base_video_path
  
  return video

In [4]:
# Get video.
def video_folder_capture(base_path):
  video_path = fetch_ucf_video(base_path)
  return video_path

In [5]:
def fetch_ucf_action_list():
  cache_list = []
  video_list = list_ucf_videos()
  for path in video_list:
    action_dir = path.split('_')[1]
    if action_dir == "HandstandPushups":
      action_dir = "HandStandPushups"
    cache_list.append(action_dir)
  cache_list = set(cache_list)
  cache_list = list(cache_list)
  cache_list.sort()
  return cache_list

In [6]:
action_list = fetch_ucf_action_list()
print(action_list)
print(action_list[0:50])
print(action_list[50:101])

['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BlowingCandles', 'BodyWeightSquats', 'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'BreastStroke', 'BrushingTeeth', 'CleanAndJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen', 'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 'FrisbeeCatch', 'FrontCrawl', 'GolfSwing', 'Haircut', 'HammerThrow', 'Hammering', 'HandStandPushups', 'HandstandWalking', 'HeadMassage', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow', 'JugglingBalls', 'JumpRope', 'JumpingJack', 'Kayaking', 'Knitting', 'LongJump', 'Lunges', 'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars', 'PizzaTossing', 'PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'P

In [7]:
def Fixed_list_ucf(classify = "Fixed",div = 0):
    cache_list = []
    action_list = fetch_ucf_action_list()
#     if div == 0:
#         action_list = action_list[0:50]
#     else:
#         action_list = action_list[50:101]

    classify_dir = os.path.join(r"C:\Users\hikar\Documents\Classified_UCF101",classify+"_Camera")
    for action in action_list:
        action_path = os.path.join(classify_dir,action)
        each_video_list = os.listdir(action_path)
        cache_list.append(each_video_list)
    return np.hstack(cache_list)

In [8]:
def crop_square(frame):
    height,width  = frame.shape[0:2]
    min_dim = min(height, width)
    start_height = 0
    start_width = (max(height, width)-min(height, width))//2
    
    return frame[start_height:start_height+min_dim,start_width:start_width+min_dim]

In [9]:
def capture_frames_from_video(videofile,classify = "Fixed"):

    action_dir = os.path.basename(os.path.dirname(videofile))
    
    vidcap = cv2.VideoCapture(videofile)
    success,image = vidcap.read()
    count = 0
    file_name  = os.path.basename(os.path.splitext(videofile)[0])#v_VolleyballSpiking_g01_c01を取り出す

    
    
    if not os.path.isdir(os.path.join(r"C:\Users\hikar\Documents\capture_all_croped" , classify , action_dir)):
        os.makedirs(os.path.join(r"C:\Users\hikar\Documents\capture_all_croped" , classify , action_dir))
        print(os.path.join(r"C:\Users\hikar\Documents\capture_all_croped" , classify , action_dir))
        
    while success:
        if not os.path.isdir(os.path.join(r"C:\Users\hikar\Documents\capture_all_croped" , classify , action_dir, "frames_%s" % (file_name))):
            os.makedirs(os.path.join(r"C:\Users\hikar\Documents\capture_all_croped" , classify , action_dir, "frames_%s" % (file_name)))
        
        croped_image = crop_square(image)
        resized_image = cv2.resize(croped_image,(64,64))
        
        
        if not os.path.isfile(os.path.join(r"C:\Users\hikar\Documents\capture_all_croped" , classify , action_dir, "frames_%s\%s.png" % (file_name,str(count).zfill(6)))):
            cv2.imwrite(os.path.join(r"C:\Users\hikar\Documents\capture_all_croped" , classify , action_dir, "frames_%s\%s.png" % (file_name,str(count).zfill(6))), resized_image)
     
        success,image = vidcap.read()
        count += 1
        
    return vidcap.get(cv2.CAP_PROP_FPS)

In [10]:
#fetch_list = list_ucf_videos()
classify = "Fixed" #Unfixed#Fixed
fetch_list = Fixed_list_ucf(classify,1)#0 or 1

for fetch_path in tqdm(fetch_list):
  # try:
  #   drive.mount('/content/drive', force_remount=True)
  # except:
  #   pass
  fetch_path = path_modify(fetch_path)
  video_path = video_folder_capture(fetch_path)
  capture_frames_from_video(video_path, classify)
print("completed")

  0%|          | 0/6831 [00:00<?, ?it/s]

C:\Users\hikar\Documents\capture_all_croped\Fixed\ApplyEyeMakeup
C:\Users\hikar\Documents\capture_all_croped\Fixed\ApplyLipstick
C:\Users\hikar\Documents\capture_all_croped\Fixed\Archery
C:\Users\hikar\Documents\capture_all_croped\Fixed\BabyCrawling
C:\Users\hikar\Documents\capture_all_croped\Fixed\BalanceBeam
C:\Users\hikar\Documents\capture_all_croped\Fixed\BandMarching
C:\Users\hikar\Documents\capture_all_croped\Fixed\BaseballPitch
C:\Users\hikar\Documents\capture_all_croped\Fixed\Basketball
C:\Users\hikar\Documents\capture_all_croped\Fixed\BasketballDunk
C:\Users\hikar\Documents\capture_all_croped\Fixed\BenchPress
C:\Users\hikar\Documents\capture_all_croped\Fixed\Biking
C:\Users\hikar\Documents\capture_all_croped\Fixed\Billiards
C:\Users\hikar\Documents\capture_all_croped\Fixed\BlowDryHair
C:\Users\hikar\Documents\capture_all_croped\Fixed\BlowingCandles
C:\Users\hikar\Documents\capture_all_croped\Fixed\BodyWeightSquats
C:\Users\hikar\Documents\capture_all_croped\Fixed\Bowling
C:\Us

In [11]:
u_list = os.listdir(r"C:\Users\hikar\Documents\Classified_UCF101\Fixed_Camera")
num = 0
count = 0
for i in u_list:
    num = len(os.listdir(os.path.join(r"C:\Users\hikar\Documents\Classified_UCF101\Fixed_Camera",i)))
    count += num
print(count)


u_list = os.listdir(r"C:\Users\hikar\Documents\Classified_UCF101\Unfixed_Camera")
num = 0
count = 0
for i in u_list:
    num = len(os.listdir(os.path.join(r"C:\Users\hikar\Documents\Classified_UCF101\Unfixed_Camera",i)))
    count += num
print(count)

6831
6489


## exp

In [12]:
img1=r"C:\Users\hikar\Documents\capture_all_frame\Fixed\ApplyEyeMakeup\frames_v_ApplyEyeMakeup_g01_c01\000000.png"
image = cv2.imread(img1)
croped_image = crop_square(image)
resized_image = cv2.resize(croped_image,(64,64))

In [13]:
image[:,40:280,:]

array([[[ 81,  65,  63],
        [ 81,  65,  63],
        [ 80,  64,  62],
        ...,
        [206, 194, 237],
        [204, 194, 237],
        [204, 194, 237]],

       [[ 81,  65,  63],
        [ 81,  65,  63],
        [ 80,  64,  62],
        ...,
        [206, 194, 237],
        [204, 194, 237],
        [204, 194, 237]],

       [[ 81,  65,  63],
        [ 81,  65,  63],
        [ 80,  64,  62],
        ...,
        [206, 194, 237],
        [204, 194, 237],
        [204, 194, 237]],

       ...,

       [[ 15,   8,  14],
        [ 21,  14,  20],
        [ 23,  17,  20],
        ...,
        [ 99,  96, 118],
        [102,  99, 121],
        [103, 100, 122]],

       [[  5,   0,   4],
        [  9,   2,   8],
        [ 43,  37,  40],
        ...,
        [ 99,  96, 118],
        [102,  99, 121],
        [103, 100, 122]],

       [[ 20,  13,  19],
        [ 20,  13,  19],
        [ 64,  58,  61],
        ...,
        [ 99,  96, 118],
        [102,  99, 121],
        [103, 100, 122]]

In [14]:
croped_image

array([[[ 81,  65,  63],
        [ 81,  65,  63],
        [ 80,  64,  62],
        ...,
        [206, 194, 237],
        [204, 194, 237],
        [204, 194, 237]],

       [[ 81,  65,  63],
        [ 81,  65,  63],
        [ 80,  64,  62],
        ...,
        [206, 194, 237],
        [204, 194, 237],
        [204, 194, 237]],

       [[ 81,  65,  63],
        [ 81,  65,  63],
        [ 80,  64,  62],
        ...,
        [206, 194, 237],
        [204, 194, 237],
        [204, 194, 237]],

       ...,

       [[ 15,   8,  14],
        [ 21,  14,  20],
        [ 23,  17,  20],
        ...,
        [ 99,  96, 118],
        [102,  99, 121],
        [103, 100, 122]],

       [[  5,   0,   4],
        [  9,   2,   8],
        [ 43,  37,  40],
        ...,
        [ 99,  96, 118],
        [102,  99, 121],
        [103, 100, 122]],

       [[ 20,  13,  19],
        [ 20,  13,  19],
        [ 64,  58,  61],
        ...,
        [ 99,  96, 118],
        [102,  99, 121],
        [103, 100, 122]]

In [15]:
resized_image.shape

(64, 64, 3)

In [16]:
cv2.imwrite(r"C:\Users\hikar\Documents\example_fol\sample.png",resized_image)

True